In [5]:
import os
import numpy as np
import pandas as pd

import image_acquisition.hal_configs as hc

NameError: name 'Optional' is not defined

# Generating hal config files for MERFISH runs

    This notebook is used to generate the shutter sequence to be imaged in each FOv
    then use that information to generate
        - the frame table, with the information for the shutter sequence, used to create config files
        - the Hal shutter file
        - the Hal config file
        - A printed version of the shutter sequence formated for clarity 
        - A grafical depiction of the shutter sequence
        
        
    all z positions are distance in microns from the locked focus, typically the coverslip
        
    a shutter sequence is determined by:
        - bead_z    :  z position where to take the bead sequence
                       The default is 
                           bead_z = 0
        
        - bead_seq  :  sequence of colors to acquire at the z_bead position to image the fiducial markers
                       The default is 
                           bead_seq = [488, np.nan]
                       that is, take a single 488 image at the coverlip position, then an empty frame
                       frame=0 is the beads
                           
        - color_seq :  sequence of colors to acquire at every z-position
                       The default is  
                           color_seq = [650,750]
                       if a empty frame is needed, us np.nan as the color, for example
                           color_seq = [np.nan, 650,750]
                       will take an empty frame at the beginning of each z position
                           
        - end_seq   :  aquisition before moving to a new FOV, typically, for going back to bead_z
                       The default is 
                           end_seq = [np.nan, np.nan]    
                           
        - z_min     :  first Z position at which acquire data 
        
        - z_max     :  last Z position at with acquire data
        
        - z_step    :  separation between z positions
        
        - z_pos     : Z positions to acquire, typically np.arange(z_min,z_max+z_step,z_step) 

In [2]:
data_dir        = f'../data'
input_dir       = f'{data_dir}/input'
output_dir      = f'{data_dir}/output'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# Define shutter sequence (560, 650)

    set parameters that fully define the shutter sequence

In [ ]:
bead_z    = 0                       # z position where to take the bead sequence
bead_seq  = [405, np.nan]           # color sequence to acquire at the z_bead position 
color_seq = [560,    650]           # color sequence to acquire at every position in z_pos 
end_seq   = [np.nan, np.nan]        # color sequence to acquire at the z_bead position after imaging all frames     
z_min     = 1                       # first Z position at which acquire data 
z_max     = 20                      # last Z position at with acquire data
z_step    = 0.5                     # separation between z positions
z_pos     = np.arange(z_min,        # Z positions to acquire 
                      z_max+z_step,
                      z_step)      

In [ ]:
# generate frame table used to create shutter file and hal config
frame_table = get_frame_table(bead_z , bead_seq, color_seq , end_seq, z_pos, microscope='MF3')

# get a succint name for color sequence
color_seq_name = get_color_sequence_name(frame_table)

# print the name (or id) for color sequence
print(f'color sequence name: {color_seq_name}\n\n')

# print frame table in format simple to see
print_frame_table(frame_table)

# save frame table to disk
frame_table.to_csv(f'{output_dir}/{color_seq_name}_frame_table.csv')

# frame_table.head()

In [ ]:
# use frame table to create the shutter file
shutter_file_path = f'{output_dir}/{color_seq_name}_shutter.xml'
create_shutter_file(frame_table, shutter_file_path)

# read the created shutter file and display
print_xml_raw(shutter_file_path)

In [ ]:
# create the hal config file from a reference file
prefix = 'hal-config-mf3-epi'
hal_config_path = create_hal_config(prefix, frame_table, xml_dir=input_dir, output_dir=output_dir)

# read the created hal file and display
print_xml_raw(hal_config_path)

In [ ]:
# recovered_df = read_shutter_file_to_frame_table(shutter_file_path, microscope='MF3')
# print_frame_table(recovered_df)

In [ ]:
# shutter_df = read_shutter_file_to_frame_table(shutter_file_path, microscope='MF3')
visualize_shutter_sequence(frame_table, 
                           title="frame table", 
                           savepath=f'{output_dir}/{color_seq_name}_shutter_sequence.png')

# Define shutter sequence (405, 488)
    set parameters that fully define the shutter sequence

In [ ]:
bead_z    = 0                       # z position where to take the bead sequence
bead_seq  = [405, np.nan]           # color sequence to acquire at the z_bead position 
color_seq = [405,    488]           # color sequence to acquire at every position in z_pos 
end_seq   = [np.nan, np.nan]        # color sequence to acquire at the z_bead position after imaging all frames     
z_min     = 1                       # first Z position at which acquire data 
z_max     = 20                      # last Z position at with acquire data
z_step    = 0.5                     # separation between z positions
z_pos     = np.arange(z_min,        # Z positions to acquire 
                      z_max+z_step,
                      z_step)      

In [ ]:
# generate frame table used to create shutter file and hal config
frame_table = get_frame_table(bead_z , bead_seq, color_seq , end_seq, z_pos, microscope='MF3')

# get a succint name for color sequence
color_seq_name = get_color_sequence_name(frame_table)

# print the name (or id) for color sequence
print(f'color sequence name: {color_seq_name}\n\n')

# print frame table in format simple to see
print_frame_table(frame_table)

# save frame table to disk
frame_table.to_csv(f'{output_dir}/{color_seq_name}_frame_table.csv')

# frame_table.head()

In [ ]:
# use frame table to create the shutter file
shutter_file_path = f'{output_dir}/{color_seq_name}_shutter.xml'
create_shutter_file(frame_table, shutter_file_path)

# read the created shutter file and display
print_xml_raw(shutter_file_path)

In [ ]:
# create the hal config file from a reference file
prefix = 'hal-config-mf3-epi'
hal_config_path = create_hal_config(prefix, frame_table, xml_dir=input_dir, output_dir=output_dir)

# read the created hal file and display
print_xml_raw(hal_config_path)

In [ ]:
# shutter_df = read_shutter_file_to_frame_table(shutter_file_path, microscope='MF3')
visualize_shutter_sequence(frame_table, 
                           title="frame table", 
                           savepath=f'{output_dir}/{color_seq_name}_shutter_sequence.png')